In [8]:
import pyvisa as pyv
import csv
import struct
import os
import redis

class WaveformGenerator:
    def __init__(self, resource_string: str, redis_host: str = 'localhost', redis_port: int = 6379):
        self.rm = pyv.ResourceManager()
        self.device = self.rm.open_resource(resource_string)
        response = self.device.query('*IDN?')
        print("Instrument ID:", response)
        self.redis_client = redis.StrictRedis(host=redis_host, port=redis_port, decode_responses=True)

    def send_command(self, command: str) -> None:
        print(f"Sending command: {command}")
        self.device.write(command)

    def query_command(self, command: str) -> str:
        print(f"Querying command: {command}")
        return self.device.query(command)

    def set_waveform_type(self, channel: int, waveform_type: str) -> None:
        self.send_command(f'C{channel}:BSWV WVTP,{waveform_type}')

    def set_frequency(self, channel: int, frequency: float) -> None:
        self.send_command(f'C{channel}:BSWV FRQ,{frequency}')

    def set_amplitude(self, channel: int, amplitude: float) -> None:
        self.send_command(f'C{channel}:BSWV AMP,{amplitude}')

    def set_phase(self, channel: int, phase: float) -> None:
        self.send_command(f'C{channel}:BSWV PHSE,{phase}')

    def set_offset(self, channel: int, offset: float) -> None:
        self.send_command(f'C{channel}:BSWV OFST,{offset}')

    def start_waveform(self, channel: int) -> None:
        self.send_command(f'C{channel}:OUTP ON')

    def stop_waveform(self, channel: int) -> None:
        self.send_command(f'C{channel}:OUTP OFF')

    def set_arbitrary_waveform_by_name(self, channel: int, name: str) -> None:
        self.send_command(f'C{channel}:ARWV NAME,{name}')

    def query_arbitrary_waveform(self, channel: int) -> str:
        return self.query_command(f'C{channel}:ARWV?')

    def convert_csv_to_binary(self, csv_filename: str, bin_filename: str) -> None:
        if not os.path.isfile(csv_filename):
            raise FileNotFoundError(f"No such file: '{csv_filename}'")
        with open(csv_filename, 'r') as csvfile, open(bin_filename, 'wb') as binfile:
            reader = csv.reader(csvfile)
            next(reader)  # Skip the header row
            for row in reader:
                float_row = list(map(float, row))
                bin_row = struct.pack('f' * len(float_row), *float_row)
                binfile.write(bin_row)
        print(f"Converted {csv_filename} to {bin_filename}")

    def save_binary_waveform_to_device(self, bin_filename: str) -> None:
        waveform_name = os.path.splitext(os.path.basename(bin_filename))[0]  # Use filename without extension as the waveform name
        with open(bin_filename, 'rb') as binfile:
            data = binfile.read()
            self.device.write_binary_values(f'C1:WVDT WVNM,{waveform_name},WAVEDATA,', data, datatype='B')
        print(f"Binary waveform {bin_filename} saved to device as {waveform_name}.")

    def upload_and_generate_waveform(self, channel: int, csv_filename: str):
        bin_filename = os.path.splitext(csv_filename)[0] + '.bin'
        self.convert_csv_to_binary(csv_filename, bin_filename)
        self.save_binary_waveform_to_device(bin_filename)
        self.set_arbitrary_waveform_by_name(channel, os.path.splitext(os.path.basename(csv_filename))[0])
        self.start_waveform(channel)
        print(f"Waveform from {csv_filename} is now being generated on channel {channel}")

    def get_command_from_redis(self, key: str):
        return self.redis_client.get(key)

    def set_command_to_redis(self, key: str, value: str):
        self.redis_client.set(key, value)

def get_user_input(prompt: str, valid_options=None):
    while True:
        value = input(prompt).strip()
        if not valid_options or value in valid_options:
            return value
        print(f"Invalid input. Valid options are: {', '.join(valid_options)}")

def waveform_config(ip_string: str = None):
    if ip_string is None:
        connection_type = get_user_input("Enter connection type (IP/USB): ", ["IP", "USB"]).upper()
        ip_address = input("Enter IP address: ").strip()
        if connection_type == "IP":
            ip_string = f"TCPIP::{ip_address}::INSTR"
        elif connection_type == "USB":
            ip_string = f"GPIB::{ip_address}::INSTR"
        else:
            print("Invalid connection type.")
            return

    generator = WaveformGenerator(ip_string)

    while True:
        try:
            channel = int(get_user_input("Enter the channel number (1/2): ", ["1", "2"]))
            mode = get_user_input("Do you want to use Redis command or manually insert functions? (redis/manual): ", ["redis", "manual"])

            if mode == "redis":
                command_key = get_user_input("Enter the Redis key for the command: ")
                if command_key == 'upload_custom_waveform':
                    file_name = get_user_input("Enter the CSV file name for custom upload: ")
                    command = f'upload {channel} {file_name}'
                else:
                    command = generator.get_command_from_redis(command_key)
            else:
                command = ""
                waveform_type = get_user_input("Enter waveform type (e.g., sine, square): ")
                frequency = float(get_user_input("Enter frequency (Hz): "))
                amplitude = float(get_user_input("Enter amplitude (V): "))
                phase = float(get_user_input("Enter phase (degrees): "))
                offset = float(get_user_input("Enter offset (V): "))

            if command:
                print(f"Received command: {command}")
                parts = command.split()
                if parts[0] == 'upload':
                    csv_filename = parts[2]
                    if not os.path.isfile(csv_filename):
                        print(f"No such file: '{csv_filename}'")
                        continue
                    generator.upload_and_generate_waveform(channel, csv_filename)
                elif parts[0] == 'set':
                    waveform_type = parts[2]
                    generator.set_waveform_type(channel, waveform_type)
                    generator.set_frequency(channel, float(parts[3]))
                    generator.set_amplitude(channel, float(parts[4]))
                    generator.start_waveform(channel)
                elif parts[0] == 'stop':
                    generator.stop_waveform(channel)
                else:
                    print("Unknown command.")
            else:
                generator.set_waveform_type(channel, waveform_type)
                generator.set_frequency(channel, frequency)
                generator.set_amplitude(channel, amplitude)
                generator.set_phase(channel, phase)
                generator.set_offset(channel, offset)
                generator.start_waveform(channel)
                print(f"Waveform set to {waveform_type} with frequency {frequency} Hz, amplitude {amplitude} V, phase {phase} degrees, and offset {offset} V on channel {channel}")

            if get_user_input("Do you want to continue? (yes/no): ", ["yes", "no"]) != "yes":
                break

        except ValueError as e:
            print(f"Invalid input or error: {e}")

if __name__ == "__main__":
    waveform_config()


Instrument ID: Siglent Technologies,SDG6052X,SDG6XFCQ6R0831,6.01.01.36R3

Sending command: C1:BSWV WVTP,square
Sending command: C1:BSWV FRQ,1000.0
Sending command: C1:BSWV AMP,1.0
Sending command: C1:BSWV PHSE,0.0
Sending command: C1:BSWV OFST,0.0
Sending command: C1:OUTP ON
Waveform set to square with frequency 1000.0 Hz, amplitude 1.0 V, phase 0.0 degrees, and offset 0.0 V on channel 1
